
# Skillra PDA: HSE course project

Карьерный навигатор Skillra для рынка IT-вакансий. Ниже — воспроизводимый пайплайн, следующее ТЗ курса (этапы 0–4) и продуктовый контекст.



## Команда и вклад
| Участник | Вклад |
| --- | --- |
| Анастасия | Очистка данных, контроль качества булевых колонок |
| Максим | Фича-инжиниринг (стек, возраст вакансии, текстовые признаки) |
| Екатерина | EDA, визуализации, зарплатные срезы |
| Дмитрий | Персоны, skill-gap и продуктовые выводы |



## 0. Настройка окружения и загрузка артефактов
Используем функции из `src.skillra_pda`, директории и пути из `config`. Если обработанные данные отсутствуют, запускаем пайплайн `scripts/run_pipeline.py`.


In [ ]:

from pathlib import Path
import sys
import importlib
import pandas as pd

CWD = Path.cwd()
CANDIDATES = [CWD] + list(CWD.parents)
PROJECT_ROOT = next((p for p in CANDIDATES if (p / "src" / "skillra_pda").exists()), CWD)
SRC_DIR = PROJECT_ROOT / "src"

for path in {PROJECT_ROOT, SRC_DIR}:
    if str(path) not in sys.path:
        sys.path.insert(0, str(path))

from src.skillra_pda import config, io, cleaning, features, eda, viz, personas

config.ensure_directories()

if not config.CLEAN_DATA_FILE.exists() or not config.FEATURE_DATA_FILE.exists():
    import scripts.run_pipeline as run_pipeline
    run_pipeline.main()

raw_path = config.RAW_DATA_FILE
clean_path = config.CLEAN_DATA_FILE
feat_path = config.FEATURE_DATA_FILE

df_raw = io.load_raw(raw_path)
df_clean = pd.read_parquet(clean_path)
df_features = pd.read_parquet(feat_path)

df_features.head(2)



## 1. Этап 0. Данные и парсинг
Датасет собран через `parser/hse_vacancies/hh_scraper.py` (HeadHunter API). Подробный словарь полей — `docs/02_feature_dictionary_hh.md`, продуктовый контекст — `docs/01_product_context_skillra.md`.


In [ ]:

raw_overview = {
    "raw_path": str(raw_path),
    "raw_shape": df_raw.shape,
    "columns_sample": df_raw.columns.tolist()[:10],
}
raw_profile = cleaning.basic_profile(df_raw)
raw_missing_top = eda.missing_share(df_raw, top_n=10)
raw_overview, raw_profile, raw_missing_top



### Выводы по этапу 0
- Данные загружаются из `data/raw` без прямых путей в ноутбуке.
- Структура и пропуски отражены через `basic_profile` и `missing_share`, опираемся на словарь признаков.
- Дальше используем только функции из `src.skillra_pda` и готовые артефакты пайплайна.



## 2. Этап 1. Предобработка
Повторяем шаги cleaning: парсинг дат, дедупликация, обработка пропусков, нормализация булевых колонок (`salary_gross` и все флаги is_/has_/benefit_/soft_/domain_/role_). Готовые результаты берём из `data/processed/hh_clean.parquet`.


In [ ]:

clean_missing_top = eda.missing_share(df_clean, top_n=10)
salary_gross_dtype = str(df_clean.get("salary_gross", pd.Series(dtype="boolean")).dtype)
salary_gross_stats = df_clean.get("salary_gross", pd.Series(dtype="boolean")).value_counts(dropna=False)

clean_snapshot = {
    "clean_shape": df_clean.shape,
    "salary_gross_dtype": salary_gross_dtype,
    "salary_gross_counts": salary_gross_stats.to_dict(),
}
clean_snapshot, clean_missing_top



### Выводы по этапу 1
- Булевы поля приведены к `pandas.BooleanDtype`, строковые маркеры `'unknown'` удалены.
- Пропуски и выбросы обработаны централизованно в `cleaning.handle_missingness`; статус отражён в сводке выше.
- Чистый датасет хранится в `data/processed/hh_clean.parquet` для дальнейших шагов.



## 3. Этап 2. Новые признаки
Фича-инжиниринг из `features.engineer_all_features`: `vacancy_age_days`, `city_tier`, `work_mode`, `salary_bucket`, стековые агрегаты (`core_data_skills_count`, `ml_stack_count`, `tech_stack_size`), текстовые метрики (`description_len_chars/words`), продуктовые флаги (`is_junior_friendly`, `battle_experience`).


In [ ]:

key_features = [
    "vacancy_age_days",
    "city_tier",
    "work_mode",
    "salary_bucket",
    "core_data_skills_count",
    "ml_stack_count",
    "tech_stack_size",
    "description_len_chars",
    "description_len_words",
    "is_junior_friendly",
    "battle_experience",
]
available = [c for c in key_features if c in df_features.columns]

feature_snapshot = df_features[available].head(5)
feature_stats = df_features[[c for c in available if pd.api.types.is_numeric_dtype(df_features[c])]].describe().T
feature_snapshot, feature_stats



### Выводы по этапу 2
- Ключевые признаки из словаря hh добавлены и сохранены в `hh_features.parquet`.
- Стековые счётчики позволяют анализировать глубину навыков, `vacancy_age_days` — свежесть вакансии.
- Признаки junior-friendly и battle_experience готовят данные к продуктовым витринам.



## 4. Этап 3. Разведочный анализ (EDA)
Используем агрегаты из `eda.py`: зарплаты по городам/грейдам/ролям, тепловая карта навыков по грейдам, бенефиты и soft-skills, junior-friendly доли.


In [ ]:
import importlib

import src.skillra_pda.eda as eda_module
eda = importlib.reload(eda_module)
if not hasattr(eda, "salary_by_city_tier"):
    raise AttributeError("expected salary_by_city_tier in eda module; ensure local code is on sys.path")

salary_city = eda.salary_by_city_tier(df_features)
salary_grade = eda.salary_by_grade(df_features)
salary_role = eda.salary_by_primary_role(df_features)
salary_stack = eda.salary_by_stack_size(df_features)

skill_cols = [c for c in df_features.columns if c.startswith("skill_") or c.startswith("has_")]
top_skills = eda.skill_frequency(df_features, skill_cols, top_n=15).index.tolist()
skill_share = eda.skill_share_by_grade(df_features, top_skills)

benefits_grade = eda.benefits_summary_by_grade(df_features)
soft_corr = eda.soft_skills_correlation(df_features)
junior_roles = eda.junior_friendly_share(df_features, group_col="primary_role")

salary_city.head(), salary_grade.head(), salary_role.head(), salary_stack.head(), skill_share.head(), benefits_grade.head(), soft_corr.head(), junior_roles.head()



### Инсайты EDA
- Зарплатные срезы доступны по ключевым разрезам (`city_tier`, `grade`, `primary_role`, размер стека).
- Тепловая карта навыков по грейдам показывает ожидания рынка к навыкам junior/middle/senior.
- Бенефиты и soft-skills агрегированы по грейдам, есть обзор junior-friendly вакансий.



## 5. Этап 4. Визуализации
Используем готовые функции из `viz.py`, сохраняем графики в `reports/figures` (требование AGENTS). Графики покрывают зарплаты, форматы работы, навыковые тепловые карты и распределения.


In [ ]:

# Перестраховка на случай обновлений модулей
viz = importlib.reload(viz)

fig_salary_grade = viz.salary_by_grade_box(df_features)
fig_salary_role = viz.salary_by_role_box(df_features)
fig_workmode = viz.work_mode_share_by_city(df_features)
fig_salary_bar = viz.salary_mean_and_count_bar(df_features, category_col="grade")
fig_skill_heatmap = viz.heatmap_skills_by_grade(skill_share)
fig_soft_corr = viz.heatmap_soft_skills_correlation(soft_corr)
fig_vacancy_age = viz.distribution_with_boxplot(df_features, column="vacancy_age_days")

[
    fig_salary_grade,
    fig_salary_role,
    fig_workmode,
    fig_salary_bar,
    fig_skill_heatmap,
    fig_soft_corr,
    fig_vacancy_age,
]



### Выводы по графикам
- Зарплатные боксплоты и барчарты дают быструю «карту рынка» по грейдам/ролям.
- Тепловые карты показывают топ-навыки и soft-skills по сегментам, чтобы подсветить skill gap.
- Распределение `vacancy_age_days` иллюстрирует свежесть выборки.



## 6. Персоны и skill-gap
Персоны из продуктового контекста: студент (Junior DA/DS), свитчер в BI/продукт, middle аналитик. Используем `Persona` и `skill_gap_for_persona` для поиска недостающих навыков.


In [ ]:

importlib.reload(personas)
skill_cols = [c for c in df_features.columns if c.startswith("skill_") or c.startswith("has_")]

persona_list = [
    personas.DATA_STUDENT_JUNIOR_DA_DS,
    personas.CAREER_SWITCHER_BI_ANALYST,
    personas.MID_DATA_ANALYST,
]

persona_gaps = {
    persona.name: personas.skill_gap_for_persona(
        df_features,
        persona,
        skill_cols=skill_cols,
        top_k=10,
    )
    for persona in persona_list
}

{key: gap.head(10) for key, gap in persona_gaps.items()}



### Выводы по персонам
- Для каждой персоны построены топ-недостающие навыки (gap=True) с учётом целевых фильтров.
- Эти выводы ложатся в рекомендационный слой Skillra: какие навыки добрать, чтобы выйти на вакансии выбранного сегмента.



## 7. Итоги и связь с продуктом Skillra
- Покрыты этапы 0–4 ТЗ: данные, предобработка, признаки, EDA, визуализации.
- Сформирована «карта рынка» по зарплатам, навыкам, бенефитам и soft-skills.
- Продуктовый слой через персоны показывает персональные skill gap и приоритеты развития.
- Репорт может быть встроен в AI-агента Skillra для карьерных рекомендаций.



## 8. Чек-лист выполнения ТЗ
| Этап | Что сделано | Секция ноутбука |
| --- | --- | --- |
| 0. Данные | Описание источника, первичный профиль | 1 |
| 1. Предобработка | Пропуски/булевы/зарплата/дубликаты | 2 |
| 2. Новые признаки | Город, формат работы, стек, тексты, свежесть | 3 |
| 3. EDA | Зарплаты, навыки, бенефиты, soft-skills, junior-friendly | 4 |
| 4. Визуализация | Графики для зарплат/навыков/распределений | 5 |
| Продуктовый слой | Персоны, skill-gap | 6 |
